In [ ]:
# Global CSS for Voila dashboard
from IPython.display import display, HTML

display(HTML('''
<style>
/* Force white text on dark header buttons */
.header-sort-button button,
.header-sort-button .widget-button,
.header-sort-button .jupyter-button,
.widget-button,
button.jupyter-button,
.lm-Widget button,
.jp-Button button {
    color: #fff !important;
}

.header-sort-button * {
    color: #fff !important;
}

/* No gap between elements */
.no-gap {
    gap: 0 !important;
}
.no-gap > * {
    margin: 0 !important;
}

/* Center all table cell text to match column headers */
table td, table th {
    text-align: center !important;
}
</style>
'''))

# TLS adapter for legacy SSL
import ssl
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.ssl_ import create_urllib3_context

class TLSAdapter(HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        ctx = create_urllib3_context()
        ctx.check_hostname = False
        ctx.verify_mode = ssl.CERT_NONE
        ctx.set_ciphers('DEFAULT:@SECLEVEL=1')
        try:
            ctx.options |= ssl.OP_LEGACY_SERVER_CONNECT
        except AttributeError:
            pass
        kwargs['ssl_context'] = ctx
        return super().init_poolmanager(*args, **kwargs)

if not getattr(requests.Session, '_tls_patched', False):
    _orig_init = requests.Session.__init__
    def _patched_init(self, *args, **kwargs):
        _orig_init(self, *args, **kwargs)
        self.mount('https://', TLSAdapter())
        self.verify = False
    requests.Session.__init__ = _patched_init
    requests.Session._tls_patched = True

In [ ]:
%%capture
%pip install -q pandas xnat

import pandas as pd
import xnat
import os
from IPython.display import display

# Configuration
PROJECT_ID = os.environ.get('XNAT_PROJECT', 'YOUR_PROJECT_ID')

# Connect to XNAT
session = xnat.connect(
    server=os.environ.get('XNAT_HOST'),
    user=os.environ.get('XNAT_USER'),
    password=os.environ.get('XNAT_PASS'),
    verify=False
)
project = session.projects[PROJECT_ID]

In [ ]:
# Load study and series data from Discovery API endpoints
def load_from_api(session, project_id, endpoint_suffix, page_size=1000, progress_callback=None):
    """Paginate through a Discovery cohort API endpoint using keyset cursor pagination.
    
    Each record has an 'id' field. The cursor is the last seen id;
    the server returns records with id > cursor. Stops when fewer
    than page_size results are returned.
    """
    all_records = []
    cursor = 0
    while True:
        resp = session.get(
            f'/xapi/discovery/cohorts/{project_id}/{endpoint_suffix}',
            query={'pageSize': page_size, 'cursor': cursor}
        )
        records = resp.json()
        all_records.extend(records)
        if progress_callback:
            progress_callback(len(all_records))
        if len(records) != page_size:
            break
        cursor = records[-1]['id']
    return all_records

def dedupe_studies(df):
    """Deduplicate studies across all data requests.
    
    1. Dedupe by studyInstanceUid - if same study appears in multiple requests,
       keep only the latest entry (by execution time), regardless of status.
    2. Dedupe AVAILABLE by experimentLabel - if different study UIDs archived to
       the same XNAT experiment, count as one.
    """
    if df.empty:
        return df
    
    df = df.copy()
    df['status'] = df['status'].str.upper()
    
    # Sort by execution time descending (latest first)
    df = df.sort_values('_execution_time', ascending=False)
    
    # Dedupe by studyInstanceUid - keeps latest entry regardless of status
    if 'studyInstanceUid' in df.columns:
        df = df.drop_duplicates(subset='studyInstanceUid', keep='first')
    
    # Dedupe AVAILABLE by experimentLabel (different study UIDs -> same experiment)
    available = df[df['status'] == 'AVAILABLE'].copy()
    other = df[df['status'] != 'AVAILABLE'].copy()
    
    if not available.empty and 'experimentLabel' in available.columns:
        has_label = available['experimentLabel'].notna()
        available_with_label = available[has_label].drop_duplicates(subset='experimentLabel', keep='first')
        available_without_label = available[~has_label]
        available = pd.concat([available_with_label, available_without_label], ignore_index=True)
    
    return pd.concat([available, other], ignore_index=True)

try:
    # Load studies from API
    study_records = load_from_api(session, PROJECT_ID, 'studies')
    study_df = pd.DataFrame(study_records)
    
    # Rename API fields to match existing column names used by dedup/dashboard
    if not study_df.empty:
        study_df.rename(columns={
            'lastExecutionTimestamp': '_execution_time',
            'dataRequestId': '_data_request_id'
        }, inplace=True)
        study_df = dedupe_studies(study_df)
    
    # Series will be loaded lazily via the "Load Series Data" button
    series_df = pd.DataFrame()
    
except Exception as e:
    from ipywidgets import HTML
    display(HTML(f'''
        <div style="background: #fdeaea; border: 1px solid #e74c3c; border-radius: 6px; padding: 15px; margin: 10px 0; color: #721c24;">
            <strong>Error loading data requests:</strong><br>
            <code>{type(e).__name__}: {e}</code>
        </div>
    '''))
    study_df, series_df = pd.DataFrame(), pd.DataFrame()

In [ ]:
# Data Request Overview
import ipywidgets as widgets
from ipywidgets import HTML, HBox, VBox, Layout, Button, Output, Dropdown
import html as html_module
import matplotlib.pyplot as plt
import warnings
import asyncio

# Suppress xnat warnings
warnings.filterwarnings('ignore')

# Global state for series data (accessible by other cells)
scans_by_experiment = {}
series_data_loaded = False
missing_series_output = None  # Will be set below, displayed in validation cell

def clean_xsi_type(xsi_type, suffix='SessionData'):
    """Clean xsiType: strip 'xnat:' prefix and suffix, uppercase result.

    Examples:
        clean_xsi_type('xnat:mrSessionData') -> 'MR'
        clean_xsi_type('xnat:ctScanData', 'ScanData') -> 'CT'
    """
    if not xsi_type or xsi_type == 'Unknown':
        return 'Unknown'

    result = str(xsi_type)

    # Strip xnat: prefix
    if result.startswith('xnat:'):
        result = result[5:]

    # Strip the suffix (SessionData or ScanData)
    if result.endswith(suffix):
        result = result[:-len(suffix)]
        return result.upper()
    else:
        # Doesn't match pattern, return as-is
        return xsi_type

def stat_card(label, value, color, tooltip=''):
    """Create a uniformly-sized status card widget with optional tooltip."""
    title_attr = f'title="{tooltip}"' if tooltip else ''
    return HTML(f'''
        <div {title_attr} style="
            background: linear-gradient(135deg, {color}22, {color}11);
            border-left: 4px solid {color};
            border-radius: 8px;
            padding: 15px 20px;
            margin: 5px;
            min-width: 170px;
            width: 170px;
            height: 70px;
            text-align: center;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            display: flex;
            flex-direction: column;
            justify-content: center;
            cursor: help;
        ">
            <div style="font-size: 28px; font-weight: bold; color: {color};">{value:,}</div>
            <div style="font-size: 11px; color: #666; text-transform: uppercase; letter-spacing: 1px; white-space: nowrap;">{label}</div>
        </div>
    ''')

# Get experiment count and subject labels
exp_response = session.get(f'/data/projects/{PROJECT_ID}/experiments', query={'format': 'json', 'columns': 'ID,label,xsiType,subject_label'})
xnat_experiments = exp_response.json().get('ResultSet', {}).get('Result', [])
project_experiment_count = len(xnat_experiments)
xnat_exp_labels = {e['label'] for e in xnat_experiments}
exp_map = {e['label']: e['ID'] for e in xnat_experiments}
exp_xsi_map = {e['label']: e.get('xsiType', 'Unknown') or 'Unknown' for e in xnat_experiments}

# Count subjects in the project
subj_response = session.get(f'/data/projects/{PROJECT_ID}/subjects', query={'format': 'json', 'columns': 'ID,label'})
xnat_subjects = subj_response.json().get('ResultSet', {}).get('Result', [])
project_subject_count = len(xnat_subjects)

if not study_df.empty:
    study_df['status'] = study_df['status'].str.upper()
    
    # Enrich study_df with session type from XNAT experiments
    study_df['sessionType'] = study_df['experimentLabel'].apply(
        lambda x: clean_xsi_type(exp_xsi_map.get(x, ''), 'SessionData') if pd.notna(x) and x in exp_xsi_map else ''
    )
    
    total = len(study_df)
    available = (study_df['status'] == 'AVAILABLE').sum()
    errors = (study_df['status'] == 'ERROR').sum()
    rejected = (study_df['status'] == 'REJECTED').sum()
    unique_accessions = study_df['accessionNumber'].nunique() if 'accessionNumber' in study_df.columns else 0
    extra_studies = total - unique_accessions
    unique_patients = study_df['patientId'].dropna().loc[lambda s: (s != '') & (s != 'Unknown')].nunique() if 'patientId' in study_df.columns else 0
    
    # Calculate patients with missing studies
    if 'patientId' in study_df.columns:
        valid_pid_mask = study_df['patientId'].notna() & (study_df['patientId'] != '') & (study_df['patientId'] != 'Unknown')
        valid_studies = study_df[valid_pid_mask].copy()
        
        # Per-patient: total requested studies
        expected_per_patient = valid_studies.groupby('patientId').size().reset_index(name='expected')
        
        # Per-patient: studies in project (AVAILABLE and experimentLabel in XNAT)
        in_project_mask = (valid_studies['status'] == 'AVAILABLE') & \
                          valid_studies['experimentLabel'].apply(lambda x: pd.notna(x) and str(x).strip() in xnat_exp_labels)
        in_project_counts = valid_studies[in_project_mask].groupby('patientId').size().reset_index(name='in_project')
        
        # Merge and calculate missing
        patient_summary = expected_per_patient.merge(in_project_counts, on='patientId', how='left')
        patient_summary['in_project'] = patient_summary['in_project'].fillna(0).astype(int)
        patient_summary['missing'] = patient_summary['expected'] - patient_summary['in_project']
        
        # Get subject labels (first non-null per patient)
        subject_labels = valid_studies.drop_duplicates('patientId')[['patientId', 'subjectLabel']]
        patient_summary = patient_summary.merge(subject_labels, on='patientId', how='left')
        
        # Only patients with missing > 0
        missing_patients_df = patient_summary[patient_summary['missing'] > 0].sort_values('patientId').reset_index(drop=True)
        missing_patient_count = len(missing_patients_df)
        completely_missing_count = len(patient_summary[patient_summary['in_project'] == 0])
        
        unknown_patient_study_count = len(study_df[
            study_df['patientId'].isna() | (study_df['patientId'] == '') | (study_df['patientId'] == 'Unknown')
        ])
    else:
        missing_patient_count = 0
        completely_missing_count = 0
        unknown_patient_study_count = 0
        missing_patients_df = pd.DataFrame(columns=['patientId', 'subjectLabel', 'missing', 'expected'])
    
    # Calculate missing from project (AVAILABLE but not in XNAT)
    available_df = study_df[study_df['status'] == 'AVAILABLE'].copy()
    if 'experimentLabel' in available_df.columns:
        def is_missing_from_project(row):
            exp_label = row.get('experimentLabel')
            if pd.isna(exp_label) or exp_label == '' or exp_label is None:
                return True
            return str(exp_label).strip() not in xnat_exp_labels
        missing_from_project_count = available_df.apply(is_missing_from_project, axis=1).sum()
    else:
        missing_from_project_count = 0
    
    # Find accession numbers with multiple studies (for use in next cell)
    if 'accessionNumber' in study_df.columns:
        accession_counts = study_df.groupby('accessionNumber').size()
        multi_accessions = accession_counts[accession_counts > 1].index.tolist()
        multi_study_accessions_count = len(multi_accessions)
        
        if multi_accessions:
            multi_study_df = study_df[study_df['accessionNumber'].isin(multi_accessions)].copy()
            multi_study_df = multi_study_df.sort_values(['accessionNumber', 'studyInstanceUid'])
        else:
            multi_study_df = pd.DataFrame()
    else:
        multi_study_accessions_count = 0
        multi_study_df = pd.DataFrame()
    
    # Main header
    header = HTML('<h1 style="margin: 0 0 20px 5px; color: #333; font-size: 32px; font-weight: 600;">Data Request Review</h1>')
    
    # Patient cards (first row)
    patient_cards = [
        stat_card('Patients In Project', project_subject_count, '#1abc9c', 'Unique subjects in XNAT project'),
        stat_card('Patients Requested', unique_patients, '#8e44ad', 'Unique patient IDs across all requested studies'),
        stat_card('Missing Patients', completely_missing_count, '#e74c3c', f'{completely_missing_count} patients completely missing from project'),
    ]
    
    # Study cards (second row)
    study_cards = [
        stat_card('Studies In Project', project_experiment_count, '#1abc9c', 'Actual studies in the project'),
        stat_card('Studies Requested', total, '#8e44ad', 'Unique Study Instance UIDs'),
        stat_card('Accessions Requested', unique_accessions, '#9b59b6', 'Unique Accession Numbers'),
        stat_card('Additional Studies', extra_studies, '#3498db', f'{extra_studies} additional studies found in PACS'),
        stat_card('Missing Studies', errors + missing_from_project_count + rejected, '#e74c3c', 'Error, missing from project, or excluded studies'),
    ]
    
    patient_row = HBox(patient_cards, layout=Layout(flex_flow='row wrap'))
    study_row = HBox(study_cards, layout=Layout(flex_flow='row wrap'))
    
    display(VBox([header, patient_row, study_row], layout=Layout(padding='10px')))
    

    
    # ==================== SESSION TYPES PIE CHART (loads immediately) ====================
    available_in_xnat = available_df[available_df['experimentLabel'].isin(exp_map.keys())]
    session_types_from_xnat = []
    
    for _, row in available_in_xnat.iterrows():
        exp_label = row.get('experimentLabel', '')
        if pd.isna(exp_label) or exp_label not in exp_map:
            continue
        xsi_type = exp_xsi_map.get(exp_label, 'Unknown')
        cleaned_type = clean_xsi_type(xsi_type, 'SessionData')
        session_types_from_xnat.append(cleaned_type)
    
    has_session_types = len(session_types_from_xnat) > 0
    
    # Create charts display (single HTML widget for both charts side by side)
    _session_b64 = ''
    if has_session_types:
        from matplotlib.figure import Figure as _Fig
        from matplotlib.backends.backend_agg import FigureCanvasAgg as _Agg
        import io as _io, base64 as _b64
        _sfig = _Fig(figsize=(6, 4.5))
        _Agg(_sfig)
        _sax = _sfig.add_subplot(111)
        
        session_type_counts = pd.Series(session_types_from_xnat).value_counts()
        total_sessions = len(session_types_from_xnat)
        colors = plt.cm.tab20(range(len(session_type_counts)))
        
        wedges, _ = _sax.pie(
            session_type_counts.values,
            startangle=90,
            colors=colors,
            wedgeprops=dict(width=0.5, edgecolor='white')
        )
        
        _sax.text(0, 0, f'{total_sessions:,}\nSessions', ha='center', va='center', 
                fontsize=14, fontweight='bold', color='#333')
        
        legend_labels = [f'{stype} - {count:,} ({count/total_sessions*100:.1f}%)' 
                         for stype, count in zip(session_type_counts.index, session_type_counts.values)]
        _sax.legend(wedges, legend_labels, title='Type', loc='center left', 
                  bbox_to_anchor=(1.05, 0.5), fontsize=9, title_fontsize=10)
        _sax.set_title('Session Types', fontsize=14, fontweight='bold', pad=15)
        
        _sfig.tight_layout()
        _sbuf = _io.BytesIO()
        _sfig.savefig(_sbuf, format='png', bbox_inches='tight', dpi=100)
        _sbuf.seek(0)
        _session_b64 = _b64.b64encode(_sbuf.read()).decode('ascii')
    
    charts_html = HTML()
    if _session_b64:
        charts_html.value = f'<div style="display:flex;gap:20px;align-items:flex-start;"><img src="data:image/png;base64,{_session_b64}"></div>'
    display(charts_html)
    
    # ==================== SERIES ANALYSIS SECTION ====================
    XNAT_HOST = os.environ.get('XNAT_HOST', '')
    
    load_series_btn = Button(
        description='Load Series Data',
        button_style='warning',
        tooltip='Fetch series data from XNAT to view scan type breakdown and missing series analysis',
        icon='download',
        layout=Layout(margin='0 15px 0 0')
    )
    
    series_output = Output()
    progress_html = HTML('<span style="color: #856404;">Click to load series data for scan type breakdown and missing series analysis</span>')
    
    # Yellow warning banner with button inside
    series_banner_content = HBox([load_series_btn, progress_html], layout=Layout(align_items='center'))
    series_note = HTML('<span style="color: #856404; font-size: 12px; opacity: 0.8;">Note: Loading series data can take several minutes.</span>')
    series_banner = VBox([series_banner_content, series_note], layout=Layout(
        background='#fff8e6',
        border='1px solid #f39c12',
        border_radius='10px',
        padding='15px 20px',
        margin='10px 5px',
        width='1500px'
    ))
    
    # Missing series output (shown after loading, displayed in validation cell at bottom)
    missing_series_output = VBox()
    
    # SOP classes excluded by site anonymization script
    EXCLUDED_SOP_CLASSES = {
        '1.2.840.10008.5.1.4.1.1.7': 'Secondary Capture Image',
        '1.2.840.10008.5.1.4.1.1.7.1': 'Multi-frame Single Bit SC Image',
        '1.2.840.10008.5.1.4.1.1.7.2': 'Multi-frame Grayscale Byte SC Image',
        '1.2.840.10008.5.1.4.1.1.7.3': 'Multi-frame Grayscale Word SC Image',
        '1.2.840.10008.5.1.4.1.1.7.4': 'Multi-frame True Color SC Image',
        '1.2.840.10008.5.1.4.1.1.104.1': 'Encapsulated PDF',
        '1.2.840.10008.5.1.4.1.1.104.2': 'Encapsulated CDA',
    }
    
    def sop_with_tooltip(sop_uid):
        sop_uid = str(sop_uid).strip() if sop_uid else ''
        if not sop_uid or sop_uid == 'nan':
            return ''
        if sop_uid in EXCLUDED_SOP_CLASSES:
            name = EXCLUDED_SOP_CLASSES[sop_uid]
            return f'<span title="{html_module.escape(sop_uid)}" style="cursor: help;">{html_module.escape(name)}</span>'
        return sop_uid
    
    async def _load_series_async():
        global scans_by_experiment, series_data_loaded
        loop = asyncio.get_event_loop()
        
        total_experiments = len(exp_map)
        
        progress_html.value = '<span style="color: #856404;">Connecting to XNAT...</span>'
        
        try:
            import logging
            logging.getLogger('xnat').setLevel(logging.ERROR)
            
            new_session = await loop.run_in_executor(None, lambda: xnat.connect(
                server=os.environ.get('XNAT_HOST'),
                user=os.environ.get('XNAT_USER'),
                password=os.environ.get('XNAT_PASS'),
                verify=False
            ))
        except Exception as e:
            progress_html.value = f'<span style="color: #c00;">Failed to connect: {e}</span>'
            return
        
        # Fetch scan data in background thread
        def _fetch_scans():
            errors_list = []
            exp_items = list(exp_map.items())
        
            for i, (exp_label, exp_id) in enumerate(exp_items):
                progress_html.value = f'<span style="color: #856404;">Loading series data from experiment {i + 1:,} / {total_experiments:,}...</span>'
            
                try:
                    scans_response = new_session.get(f'/data/experiments/{exp_id}/scans', query={'format': 'json', 'columns': 'ID,UID,series_description,xsiType'})
                    scans_data = scans_response.json().get('ResultSet', {}).get('Result', [])
                    scans_by_experiment[exp_label] = [
                        {
                            'id': scan.get('ID', ''),
                            'uid': scan.get('UID', ''),
                            'series_description': scan.get('series_description', ''),
                            'xsiType': scan.get('xsiType', 'Unknown') or 'Unknown'
                        }
                        for scan in scans_data
                    ]
                except Exception as e:
                    scans_by_experiment[exp_label] = []
                    if len(errors_list) < 3:
                        errors_list.append(f"{exp_label}: {type(e).__name__}: {e}")
        
        await loop.run_in_executor(None, _fetch_scans)
        series_data_loaded = True
        
        # Build scan type data
        all_project_scans = []
        for exp_label, scans in scans_by_experiment.items():
            for scan in scans:
                cleaned_type = clean_xsi_type(scan['xsiType'], 'ScanData')
                all_project_scans.append({'type': cleaned_type})
        
        exps_with_scans = sum(1 for scans in scans_by_experiment.values() if scans)
        total_scans = len(all_project_scans)
        
        # Update button to show loaded state
        load_series_btn.disabled = True
        load_series_btn.description = 'Series Loaded'
        load_series_btn.button_style = ''
        load_series_btn.icon = 'check'
        load_series_btn.style.button_color = '#6c757d'
        
        # Create scan types chart (using Agg renderer directly to avoid backend issues)
        if all_project_scans:
          try:
            from matplotlib.figure import Figure
            from matplotlib.backends.backend_agg import FigureCanvasAgg
            import io, base64
            
            fig = Figure(figsize=(6, 4.5))
            FigureCanvasAgg(fig)
            ax = fig.add_subplot(111)
            
            scan_type_counts = pd.DataFrame(all_project_scans)['type'].value_counts()
            total_series = len(all_project_scans)
            colors = plt.cm.tab20(range(len(scan_type_counts)))
            
            wedges, _ = ax.pie(
                scan_type_counts.values,
                startangle=90,
                colors=colors,
                wedgeprops=dict(width=0.5, edgecolor='white')
            )
            
            ax.text(0, 0, f'{total_series:,}\nScans', ha='center', va='center', 
                    fontsize=14, fontweight='bold', color='#333')
            
            legend_labels = [f'{stype} - {count:,} ({count/total_series*100:.1f}%)' 
                             for stype, count in zip(scan_type_counts.index, scan_type_counts.values)]
            ax.legend(wedges, legend_labels, title='Type', loc='center left', 
                      bbox_to_anchor=(1.05, 0.5), fontsize=9, title_fontsize=10)
            ax.set_title('Scan Types', fontsize=14, fontweight='bold', pad=15)
            
            fig.tight_layout()
            buf = io.BytesIO()
            fig.savefig(buf, format='png', bbox_inches='tight', dpi=100)
            buf.seek(0)
            png_b64 = base64.b64encode(buf.read()).decode('ascii')
            charts_html.value = f'<div style="display:flex;gap:20px;align-items:flex-start;"><img src="data:image/png;base64,{_session_b64}"><img src="data:image/png;base64,{png_b64}"></div>'
          except Exception as _chart_err:
            charts_html.value = f'<div style="display:flex;gap:20px;align-items:flex-start;"><img src="data:image/png;base64,{_session_b64}"><div style="color:red;padding:10px;">Chart error: {type(_chart_err).__name__}: {_chart_err}</div></div>'
        
        
        # ==================== BUILD MISSING SERIES WIDGET ====================
        XNAT_HOST = os.environ.get('XNAT_HOST', '')
        studies_with_missing = []
        
        available = study_df[study_df['status'] == 'AVAILABLE']
        
        # Load series from Discovery API for missing series analysis
        # page_size must match server MAX_PAGE_SIZE (1000) for pagination to work
        progress_html.value = '<span style="color: #856404;">Loading series data from Discovery API...</span>'
        def _fetch_series():
            try:
                series_records = load_from_api(new_session, PROJECT_ID, 'series',
                    progress_callback=lambda n: setattr(progress_html, 'value',
                        f'<span style="color: #856404;">Loading series data from Discovery API... ({n:,} records)</span>'))
                return pd.DataFrame(series_records)
            except Exception as e:
                return pd.DataFrame()
        api_series_df = await loop.run_in_executor(None, _fetch_series)
        series_by_exp = api_series_df.groupby('experimentLabel') if not api_series_df.empty and 'experimentLabel' in api_series_df.columns else None
        
        progress_html.value = '<span style="color: #856404;">Analyzing missing series...</span>'
        
        try:
            for _, row in available.iterrows():
                exp_label = row.get('experimentLabel', '')
                if pd.isna(exp_label) or exp_label not in exp_map:
                    continue
                
                xnat_scans = scans_by_experiment.get(exp_label, [])
                actual_count = len(xnat_scans)
                xnat_series_uids = {scan.get('uid', '').strip() for scan in xnat_scans if scan.get('uid')}
                
                if series_by_exp is not None and exp_label in series_by_exp.groups:
                    expected = series_by_exp.get_group(exp_label)
                    expected_count = len(expected)

                    def is_missing(r):
                        uid = str(r.get('seriesInstanceUid', '')).strip()
                        if not uid or uid == 'nan':
                            return True
                        return uid not in xnat_series_uids

                    all_missing_series = expected[expected.apply(is_missing, axis=1)].copy()

                    if len(all_missing_series) > 0:
                        excluded_series = all_missing_series[all_missing_series['sopClassUid'].isin(EXCLUDED_SOP_CLASSES.keys())].copy()
                        actually_missing = all_missing_series[~all_missing_series['sopClassUid'].isin(EXCLUDED_SOP_CLASSES.keys())].copy()
                        
                        excluded_count = len(excluded_series)
                        unexplained_count = len(actually_missing)
                        
                        if unexplained_count > 0:
                            study_uid = row.get('studyInstanceUid', '')
                            if pd.isna(study_uid):
                                study_uid = ''
                            studies_with_missing.append({
                                'experimentLabel': exp_label,
                                'accessionNumber': row.get('accessionNumber', ''),
                                'studyInstanceUid': study_uid,
                                'expected': expected_count,
                                'actual': actual_count,
                                'missing': excluded_count + unexplained_count,
                                'excluded_sop': excluded_count,
                                'unexplained': unexplained_count,
                                'excluded_series': excluded_series,
                                'actually_missing': actually_missing
                            })
        except Exception as e:
            progress_html.value = f'<span style="color: #c00;">Error analyzing series: {type(e).__name__}: {e}</span>'
            return
        
        # Update banner to final loaded state
        series_count = len(api_series_df) if not api_series_df.empty else 0
        progress_html.value = f'<span style="color: #155724;">&#10003; <strong>Loaded {total_scans:,} scans from {exps_with_scans:,} experiments, {series_count:,} series from Discovery API.</strong> Scroll down to view Missing Series Analysis.</span>'
        series_banner.layout.background = '#d4edda'
        series_banner.layout.border = '1px solid #28a745'
        series_note.value = ''
        
        if studies_with_missing:
            total_missing_studies = len(studies_with_missing)
            total_excluded = sum(s['excluded_sop'] for s in studies_with_missing)
            total_unexplained = sum(s['unexplained'] for s in studies_with_missing)
                
            ms_header = HTML(f'''
                <div style="
                    background: linear-gradient(135deg, #e74c3c 0%, #c0392b 100%);
                    color: white;
                    padding: 20px;
                    border-radius: 10px 10px 0 0;
                ">
                    <h3 style="margin: 0; font-size: 18px;">Missing Series</h3>
                    <p style="margin: 5px 0 0 0; opacity: 0.9; font-size: 13px;">{total_missing_studies} studies with missing series</p>
                </div>
            ''')
                
            ms_summary = HTML(f'''
                <div style="display: flex; gap: 15px; padding: 15px 20px; background: #f8f9fa;">
                    <div style="text-align: center; padding: 10px 20px; background: white; border-radius: 8px; box-shadow: 0 1px 3px rgba(0,0,0,0.1);">
                        <div style="font-size: 24px; font-weight: bold; color: #e74c3c;">{total_missing_studies}</div>
                        <div style="font-size: 11px; color: #666; text-transform: uppercase;">Studies Affected</div>
                    </div>
                    <div style="text-align: center; padding: 10px 20px; background: white; border-radius: 8px; box-shadow: 0 1px 3px rgba(0,0,0,0.1);">
                        <div style="font-size: 24px; font-weight: bold; color: #f39c12;">{total_excluded}</div>
                        <div style="font-size: 11px; color: #666; text-transform: uppercase;">Excluded</div>
                    </div>
                    <div style="text-align: center; padding: 10px 20px; background: white; border-radius: 8px; box-shadow: 0 1px 3px rgba(0,0,0,0.1);">
                        <div style="font-size: 24px; font-weight: bold; color: #c0392b;">{total_unexplained}</div>
                        <div style="font-size: 11px; color: #666; text-transform: uppercase;">Missing</div>
                    </div>
                </div>
            ''')
                
            # Build options list (no placeholder, start with first session)
            options = []
            for s in studies_with_missing:
                label = f"{s['experimentLabel']} \u2014 {s['missing']} missing ({s['excluded_sop']} excluded, {s['unexplained']} missing)"
                options.append((label, s))
                
            # Create dropdown with first session selected
            ms_dropdown = Dropdown(options=options, value=studies_with_missing[0], layout=Layout(width='100%'))
            ms_detail_output = VBox()
                
            def on_ms_select(change):
                study = change['new']
                if study is None:
                    ms_detail_output.children = []
                    return
                    
                xnat_url = f"{XNAT_HOST}/data/archive/projects/{PROJECT_ID}/experiments/{study['experimentLabel']}"
                    
                _header_widget = HTML(f'''
                        <div style="background: white; padding: 20px;">
                            <div style="display: flex; justify-content: space-between; align-items: center;">
                                <div>
                                    <div style="font-size: 16px; font-weight: bold;">
                                        <a href="{xnat_url}" target="_blank" style="color: #e74c3c; text-decoration: none;">{study['experimentLabel']}</a>
                                    </div>
                                    <div style="font-size: 12px; color: #888; margin-top: 4px;">Accession: {study['accessionNumber']}</div>
                                    <div style="font-size: 11px; color: #aaa; margin-top: 2px;">Study Instance UID: {study['studyInstanceUid']}</div>
                                </div>
                                <div style="display: flex; gap: 10px;">
                                    <div style="text-align: center; padding: 8px 15px; background: #e8f4fd; border-radius: 6px;">
                                        <div style="font-size: 18px; font-weight: bold; color: #3498db;">{study['expected']}</div>
                                        <div style="font-size: 10px; color: #666;">Expected</div>
                                    </div>
                                    <div style="text-align: center; padding: 8px 15px; background: #e8f8e8; border-radius: 6px;">
                                        <div style="font-size: 18px; font-weight: bold; color: #27ae60;">{study['actual']}</div>
                                        <div style="font-size: 10px; color: #666;">In XNAT</div>
                                    </div>
                                    <div style="text-align: center; padding: 8px 15px; background: #fff8e6; border-radius: 6px;">
                                        <div style="font-size: 18px; font-weight: bold; color: #f39c12;">{study['excluded_sop']}</div>
                                        <div style="font-size: 10px; color: #856404;">Excluded</div>
                                    </div>
                                    <div style="text-align: center; padding: 8px 15px; background: #fdeaea; border-radius: 6px;">
                                        <div style="font-size: 18px; font-weight: bold; color: #e74c3c;">{study['unexplained']}</div>
                                        <div style="font-size: 10px; color: #721c21;">Missing</div>
                                    </div>
                                </div>
                            </div>
                        </div>
                ''')
                    
                all_missing = []
                is_excluded_flags = []
                        
                for _, r in study['excluded_series'].iterrows():
                    all_missing.append({
                        'Series UID': r.get('seriesInstanceUid', ''),
                        'Status': 'Excluded',
                        'Series Description': r.get('seriesDescription', ''),
                        'Modality': r.get('modality', ''),
                        'SOP Class': sop_with_tooltip(r.get('sopClassUid', ''))
                    })
                    is_excluded_flags.append(True)
                        
                for _, r in study['actually_missing'].iterrows():
                    all_missing.append({
                        'Series UID': r.get('seriesInstanceUid', ''),
                        'Status': 'Missing',
                        'Series Description': r.get('seriesDescription', ''),
                        'Modality': r.get('modality', ''),
                        'SOP Class': sop_with_tooltip(r.get('sopClassUid', ''))
                    })
                    is_excluded_flags.append(False)
                        
                if all_missing:
                    display_df = pd.DataFrame(all_missing)
                    def style_row(row_idx):
                        if is_excluded_flags[row_idx]:
                            return ['background-color: #fff8e6; color: #856404;'] * len(display_df.columns)
                        else:
                            return ['background-color: #fdeaea; color: #721c24;'] * len(display_df.columns)

                    styled = display_df.style.apply(lambda x: style_row(x.name), axis=1)\
                        .set_properties(**{'text-align': 'left', 'font-size': '12px', 'padding': '10px'})\
                        .set_table_styles([
                            {'selector': '', 'props': [('width', '100%'), ('border-collapse', 'collapse')]},
                            {'selector': 'th', 'props': [('background-color', '#f8f9fa'), ('font-size', '11px'), ('text-transform', 'uppercase'), ('color', '#666'), ('padding', '10px'), ('border-bottom', '2px solid #ddd')]},
                            {'selector': 'td', 'props': [('border-bottom', '1px solid #eee')]},
                        ]).hide(axis='index')

                    table_html = styled.to_html()
                    table_html = table_html.replace('&lt;span title=', '<span title=').replace('style=&quot;cursor: help;&quot;&gt;', 'style="cursor: help;">').replace('&lt;/span&gt;', '</span>').replace('&quot;', '"')
                    _table_widget = HTML(f'<div style="background: white; padding: 15px 20px;">{table_html}</div>')
                
                ms_detail_output.children = [_header_widget, _table_widget] if all_missing else [_header_widget]

            ms_dropdown.observe(on_ms_select, names='value')
                
            # Label for dropdown
            ms_dropdown_label = HTML('<div style="font-size: 12px; font-weight: bold; color: #666; margin-bottom: 5px;">Select Study</div>')
                
            ms_widget = VBox([
                ms_header,
                ms_summary,
                VBox([ms_dropdown_label, ms_dropdown], layout=Layout(padding='15px 20px', width='100%', overflow='hidden')),
                ms_detail_output
            ], layout=Layout(width='1500px', border_radius='10px', overflow='hidden', background='#f8f9fa', margin='10px 5px'))
                
            # Trigger initial display of first session
            on_ms_select({'new': studies_with_missing[0]})
            missing_series_output.children = [ms_widget]
        else:
            missing_series_output.children = [HTML('''
                <div style="background: linear-gradient(135deg, #56ab2f 0%, #a8e063 100%); color: white; padding: 20px; border-radius: 10px; text-align: center; width: 1500px; margin: 10px 5px;">
                    <h3 style="margin: 0;">All Series Complete</h3>
                    <p style="margin: 10px 0 0 0; opacity: 0.9;">No missing series detected.</p>
                </div>
            ''')]

    def on_load_series(b):
        b.disabled = True
        asyncio.ensure_future(_load_series_async())
    load_series_btn.on_click(on_load_series)
    
    display(series_banner)
    # Note: missing_series_output is displayed in the validation cell at the bottom

else:
    print("No study data available.")

In [ ]:
# Missing Patients (patients with missing studies)
from ipywidgets import HTML, HBox, VBox, Layout, Button, Output, Text
import html as html_module

if not study_df.empty and missing_patient_count > 0 and not missing_patients_df.empty:
    mp_details = []
    for _, row in missing_patients_df.iterrows():
        patient_id = str(row.get('patientId', '')).strip()
        if not patient_id or patient_id == 'nan' or patient_id == 'None':
            patient_id = 'Unknown'

        subject_label = str(row.get('subjectLabel', '')).strip()
        if not subject_label or subject_label == 'nan' or subject_label == 'None':
            subject_label = 'Unknown'

        missing_count = int(row.get('missing', 0))
        expected_count = int(row.get('expected', 0))

        mp_details.append({
            'Patient ID': patient_id,
            'Subject Label': subject_label,
            'Missing Studies': missing_count,
            'Expected Studies': expected_count,
        })

    mp_table_df = pd.DataFrame(mp_details)

    MP_PAGE_SIZE = 15
    mp_state = {'page': 0, 'sort_col': 'Missing Studies', 'sort_asc': False, 'filter': ''}

    mp_columns = [
        ('Patient ID', '30%'),
        ('Subject Label', '30%'),
        ('Missing Studies', '20%'),
        ('Expected Studies', '20%'),
    ]

    mp_filter_input = Text(placeholder='Search', layout=Layout(width='300px'))
    mp_prev_btn = Button(description='Previous', button_style='info', layout=Layout(width='100px'))
    mp_next_btn = Button(description='Next', button_style='info', layout=Layout(width='100px'))
    mp_page_label = HTML()
    mp_table_output = Output()

    mp_header_buttons = []

    def update_mp_header_buttons():
        for i, (col_name, width) in enumerate(mp_columns):
            arrow = ''
            if mp_state['sort_col'] == col_name:
                arrow = ' ↑' if mp_state['sort_asc'] else ' ↓'
            mp_header_buttons[i].description = f'{col_name}{arrow}'

    def make_mp_sort_handler(col_name):
        def handler(b):
            if mp_state['sort_col'] == col_name:
                mp_state['sort_asc'] = not mp_state['sort_asc']
            else:
                mp_state['sort_col'] = col_name
                mp_state['sort_asc'] = True
            mp_state['page'] = 0
            update_mp_header_buttons()
            render_mp_table()
        return handler

    def render_mp_table():
        mp_table_output.clear_output(wait=True)

        filtered_df = mp_table_df.copy()
        if mp_state['filter']:
            filter_pattern = mp_state['filter'].replace('*', '.*')
            mask = (
                filtered_df['Patient ID'].astype(str).str.contains(filter_pattern, case=False, regex=True, na=False) |
                filtered_df['Subject Label'].astype(str).str.contains(filter_pattern, case=False, regex=True, na=False)
            )
            filtered_df = filtered_df[mask]

        sort_col = mp_state['sort_col']
        if sort_col in ('Missing Studies', 'Expected Studies'):
            sorted_df = filtered_df.sort_values(by=sort_col, ascending=mp_state['sort_asc']).reset_index(drop=True)
        else:
            sorted_df = filtered_df.sort_values(
                by=sort_col,
                ascending=mp_state['sort_asc'],
                key=lambda x: x.str.lower() if x.dtype == 'object' else x
            ).reset_index(drop=True)

        total_pages = max(1, (len(sorted_df) + MP_PAGE_SIZE - 1) // MP_PAGE_SIZE)
        mp_state['page'] = max(0, min(mp_state['page'], total_pages - 1))
        start = mp_state['page'] * MP_PAGE_SIZE
        end = start + MP_PAGE_SIZE
        page_df = sorted_df.iloc[start:end].copy()

        filter_text = f" (filtered: {len(filtered_df)})" if mp_state['filter'] else ""
        mp_page_label.value = f'<span style="font-size: 13px; color: #666;">Page {mp_state["page"] + 1} of {total_pages} ({len(sorted_df)} patients{filter_text})</span>'

        mp_prev_btn.disabled = mp_state['page'] == 0
        mp_next_btn.disabled = mp_state['page'] >= total_pages - 1

        rows = []
        for _, r in page_df.iterrows():
            pid = html_module.escape(str(r['Patient ID']))
            slabel = html_module.escape(str(r['Subject Label']))
            missing = int(r['Missing Studies'])
            expected = int(r['Expected Studies'])

            rows.append(f'''<tr style="background-color: #fdeaea; color: #721c24;">
                <td style="padding: 8px 12px; border-bottom: 1px solid #eee; width: 30%;">{pid}</td>
                <td style="padding: 8px 12px; border-bottom: 1px solid #eee; width: 30%;">{slabel}</td>
                <td style="padding: 8px 12px; border-bottom: 1px solid #eee; width: 20%; font-weight: bold;">{missing}</td>
                <td style="padding: 8px 12px; border-bottom: 1px solid #eee; width: 20%;">{expected}</td>
            </tr>''')

        table_html = f'''<table style="width: 100%; border-collapse: collapse; font-size: 12px; table-layout: fixed; background: white; margin: 0; padding: 0;">
            <tbody>{"".join(rows)}</tbody>
        </table>'''

        with mp_table_output:
            display(HTML(table_html))

    for col_name, width in mp_columns:
        arrow = ''
        if mp_state['sort_col'] == col_name:
            arrow = ' ↑' if mp_state['sort_asc'] else ' ↓'
        btn = Button(description=f'{col_name}{arrow}')
        btn.layout = Layout(width=width, height='36px', margin='0', padding='0')
        btn.style.button_color = '#333'
        btn.add_class('header-sort-button')
        btn.on_click(make_mp_sort_handler(col_name))
        mp_header_buttons.append(btn)

    def on_mp_filter_change(change):
        mp_state['filter'] = change['new']
        mp_state['page'] = 0
        render_mp_table()

    def on_mp_prev(b):
        mp_state['page'] -= 1
        render_mp_table()

    def on_mp_next(b):
        mp_state['page'] += 1
        render_mp_table()

    mp_filter_input.observe(on_mp_filter_change, names='value')
    mp_prev_btn.on_click(on_mp_prev)
    mp_next_btn.on_click(on_mp_next)

    render_mp_table()

    # Build description with unknown patient note
    mp_desc = f'{missing_patient_count} patients with missing studies'
    completely_missing = len(mp_table_df[mp_table_df['Missing Studies'] == mp_table_df['Expected Studies']])
    if completely_missing > 0:
        mp_desc += f'. {completely_missing} patients completely missing'
    if unknown_patient_study_count > 0:
        mp_desc += f'<br><span style="opacity: 0.85;">Note: The Patient ID is unknown for {unknown_patient_study_count} studies. Please check the missing studies table.</span>'

    mp_title_widget = HTML(f'''
        <div style="
            background: linear-gradient(135deg, #e74c3c 0%, #c0392b 100%);
            color: white;
            padding: 20px;
            border-radius: 10px 10px 0 0;
        ">
            <h3 style="margin: 0; font-size: 18px;">Missing Patients</h3>
            <p style="margin: 5px 0 0 0; opacity: 0.9; font-size: 13px;">{mp_desc}</p>
        </div>
    ''')

    mp_left_controls = HBox([mp_filter_input], layout=Layout(align_items='center'))
    mp_right_controls = HBox([mp_prev_btn, mp_page_label, mp_next_btn], layout=Layout(align_items='center', gap='10px'))
    mp_controls = HBox(
        [mp_left_controls, mp_right_controls],
        layout=Layout(padding='15px 20px', justify_content='space-between', align_items='center', background='#f5f5f5', width='100%')
    )

    mp_header_row = HBox(mp_header_buttons)
    mp_header_row.layout = Layout(width='100%', margin='0', padding='0', background='#333', overflow='hidden')
    mp_header_row.add_class('no-gap')

    mp_table_wrapper = VBox([mp_table_output])
    mp_table_wrapper.layout = Layout(background='white', padding='0', margin='0', width='100%', overflow='hidden')

    mp_table_container = VBox([mp_header_row, mp_table_wrapper])
    mp_table_container.layout = Layout(background='#333', padding='0', margin='0', width='100%', overflow='hidden')
    mp_table_container.add_class('no-gap')

    mp_full_widget = VBox([
        mp_title_widget,
        mp_controls,
        mp_table_container,
    ])
    mp_full_widget.layout = Layout(width='1500px', border_radius='10px', overflow='hidden', background='#f5f5f5', margin='10px 5px')
    mp_full_widget.add_class('no-gap')

    display(mp_full_widget)

In [ ]:
# Additional Studies (accessions with multiple study UIDs)
from ipywidgets import HTML, HBox, VBox, Layout, Button, Output, Text
import ipywidgets as widgets
import html as html_module

# Get XNAT host for building links
XNAT_HOST = os.environ.get('XNAT_HOST', '')

def truncate_with_tooltip(text, max_len=50):
    """Truncate text and add tooltip if needed."""
    text = str(text)
    if len(text) <= max_len:
        return html_module.escape(text)
    truncated = text[:max_len].rsplit(' ', 1)[0] + '...'
    escaped_full = html_module.escape(text)
    escaped_truncated = html_module.escape(truncated)
    return f'<span title="{escaped_full}" style="cursor: help;">{escaped_truncated}</span>'

# CSS to fix button styling - force white text on dark background for both light/dark themes
add_button_style_css = HTML('''
<style>
.header-sort-button button,
.header-sort-button .widget-button,
.header-sort-button .jupyter-button,
.header-sort-button button .bp3-button-text,
.header-sort-button button span,
.header-sort-button button p {
    border-radius: 0 !important;
    font-weight: bold !important;
    font-size: 11px !important;
    text-transform: uppercase !important;
    border: none !important;
    border-bottom: 2px solid #555 !important;
    background-color: #333 !important;
    color: #fff !important;
}
.header-sort-button button:hover {
    background-color: #444 !important;
}
.no-gap {
    gap: 0 !important;
}
.no-gap > * {
    margin: 0 !important;
}
</style>
''')
display(add_button_style_css)

# ==================== ADDITIONAL STUDIES TABLE ====================
if not study_df.empty and extra_studies > 0 and not multi_study_df.empty:
    additional_details = []
    for _, row in multi_study_df.iterrows():
        exp_label = str(row.get('experimentLabel', '')).strip()
        if not exp_label or exp_label == 'nan' or exp_label == 'None':
            exp_label = 'Unknown'
        
        study_uid = str(row.get('studyInstanceUid', '')).strip()
        if not study_uid or study_uid == 'nan' or study_uid == 'None':
            study_uid = 'Unknown'
        
        status = row.get('status', '')
        if status == 'ERROR':
            reason = str(row.get('statusMessage', '')).strip() or 'Unknown'
        elif status == 'REJECTED':
            reason = str(row.get('statusMessage', '')).strip() or 'Rejected by site'
        else:
            reason = ''
        
        session_type = row.get('sessionType', '')
        if pd.isna(session_type):
            session_type = ''
        
        patient_id = str(row.get('patientId', '')).strip()
        if not patient_id or patient_id == 'nan' or patient_id == 'None':
            patient_id = 'Unknown'
        
        subject_label = str(row.get('subjectLabel', '')).strip()
        if not subject_label or subject_label == 'nan' or subject_label == 'None':
            subject_label = 'Unknown'
        
        additional_details.append({
            'Experiment Label': str(exp_label),
            'Patient ID': str(patient_id),
            'Subject Label': str(subject_label),
            'Session Type': str(session_type),
            'Accession Number': str(row.get('accessionNumber', '')),
            'Study UID': study_uid,
            'Status': status,
            'Reason': reason,
        })
    
    additional_table_df = pd.DataFrame(additional_details)
    
    PAGE_SIZE = 15
    add_state = {'page': 0, 'sort_col': 'Accession Number', 'sort_asc': True, 'filter': ''}
    
    # Column definitions
    add_columns = [
        ('Experiment Label', '14%'),
        ('Patient ID', '9%'),
        ('Subject Label', '9%'),
        ('Session Type', '10%'),
        ('Accession Number', '10%'),
        ('Study UID', '26%'),
        ('Status', '6%'),
        ('Reason', '16%'),
    ]
    
    add_filter_input = Text(
        placeholder='Search',
        layout=Layout(width='300px')
    )
    
    add_prev_btn = Button(description='Previous', button_style='info', layout=Layout(width='100px'))
    add_next_btn = Button(description='Next', button_style='info', layout=Layout(width='100px'))
    add_page_label = HTML()
    add_table_output = Output()
    
    # Create header buttons list first
    add_header_buttons = []
    
    def update_add_header_buttons():
        """Update header button labels with sort arrows."""
        for i, (col_name, width) in enumerate(add_columns):
            arrow = ''
            if add_state['sort_col'] == col_name:
                arrow = ' ↑' if add_state['sort_asc'] else ' ↓'
            add_header_buttons[i].description = f'{col_name}{arrow}'
    
    def make_add_sort_handler(col_name):
        def handler(b):
            if add_state['sort_col'] == col_name:
                add_state['sort_asc'] = not add_state['sort_asc']
            else:
                add_state['sort_col'] = col_name
                add_state['sort_asc'] = True
            add_state['page'] = 0
            update_add_header_buttons()
            render_additional_table()
        return handler
    
    def render_additional_table():
        add_table_output.clear_output(wait=True)
        
        # Apply filter across all columns
        filtered_df = additional_table_df.copy()
        if add_state['filter']:
            filter_pattern = add_state['filter'].replace('*', '.*')
            mask = (
                filtered_df['Experiment Label'].str.contains(filter_pattern, case=False, regex=True, na=False) |
                filtered_df['Patient ID'].str.contains(filter_pattern, case=False, regex=True, na=False) |
                filtered_df['Subject Label'].str.contains(filter_pattern, case=False, regex=True, na=False) |
                filtered_df['Session Type'].str.contains(filter_pattern, case=False, regex=True, na=False) |
                filtered_df['Accession Number'].str.contains(filter_pattern, case=False, regex=True, na=False) |
                filtered_df['Study UID'].str.contains(filter_pattern, case=False, regex=True, na=False) |
                filtered_df['Status'].str.contains(filter_pattern, case=False, regex=True, na=False) |
                filtered_df['Reason'].str.contains(filter_pattern, case=False, regex=True, na=False)
            )
            filtered_df = filtered_df[mask]
        
        sorted_df = filtered_df.sort_values(
            by=add_state['sort_col'],
            ascending=add_state['sort_asc'],
            key=lambda x: x.str.lower() if x.dtype == 'object' else x
        ).reset_index(drop=True)
        
        total_pages = max(1, (len(sorted_df) + PAGE_SIZE - 1) // PAGE_SIZE)
        add_state['page'] = max(0, min(add_state['page'], total_pages - 1))
        start = add_state['page'] * PAGE_SIZE
        end = start + PAGE_SIZE
        page_df = sorted_df.iloc[start:end].copy()
        
        filter_text = f" (filtered: {len(filtered_df)})" if add_state['filter'] else ""
        add_page_label.value = f'<span style="font-size: 13px; color: #666;">Page {add_state["page"] + 1} of {total_pages} ({len(sorted_df)} studies{filter_text})</span>'
        
        add_prev_btn.disabled = add_state['page'] == 0
        add_next_btn.disabled = add_state['page'] >= total_pages - 1
        
        # Build data rows
        rows = []
        for _, r in page_df.iterrows():
            exp_label = str(r['Experiment Label'])
            pid = html_module.escape(str(r['Patient ID']))
            slabel = html_module.escape(str(r['Subject Label']))
            sess_type = html_module.escape(str(r['Session Type']))
            acc = html_module.escape(str(r['Accession Number']))
            uid = html_module.escape(str(r['Study UID']))
            stat = html_module.escape(str(r['Status']))
            reason = truncate_with_tooltip(str(r['Reason']), 50)
            
            # Color based on status
            status_upper = str(r['Status']).upper()
            if status_upper == 'ERROR':
                bg_color = '#fdeaea'
                text_color = '#721c24'
            elif status_upper == 'REJECTED':
                bg_color = '#fff8e6'
                text_color = '#856404'
            else:
                bg_color = '#f0f7ff'
                text_color = '#2c5282'
            
            # Create experiment label with link for AVAILABLE studies
            if status_upper == 'AVAILABLE' and exp_label and exp_label != 'Unknown':
                xnat_url = f"{XNAT_HOST}/data/archive/projects/{PROJECT_ID}/experiments/{exp_label}"
                exp_display = f'<a href="{xnat_url}" target="_blank" style="color: {text_color}; text-decoration: underline;">{html_module.escape(exp_label)}</a>'
            else:
                exp_display = html_module.escape(exp_label)
            
            rows.append(f'''<tr style="background-color: {bg_color}; color: {text_color};">
                <td style="padding: 8px 12px; border-bottom: 1px solid #eee; width: 14%;">{exp_display}</td>
                <td style="padding: 8px 12px; border-bottom: 1px solid #eee; width: 9%;">{pid}</td>
                <td style="padding: 8px 12px; border-bottom: 1px solid #eee; width: 9%;">{slabel}</td>
                <td style="padding: 8px 12px; border-bottom: 1px solid #eee; width: 10%;">{sess_type}</td>
                <td style="padding: 8px 12px; border-bottom: 1px solid #eee; width: 10%;">{acc}</td>
                <td style="padding: 8px 12px; border-bottom: 1px solid #eee; width: 26%; word-break: break-all;">{uid}</td>
                <td style="padding: 8px 12px; border-bottom: 1px solid #eee; width: 6%;">{stat}</td>
                <td style="padding: 8px 12px; border-bottom: 1px solid #eee; width: 16%;">{reason}</td>
            </tr>''')
        
        table_html = f'''<table style="width: 100%; border-collapse: collapse; font-size: 12px; table-layout: fixed; background: white; margin: 0; padding: 0;">
            <tbody>{"".join(rows)}</tbody>
        </table>'''
        
        with add_table_output:
            display(HTML(table_html))
    
    # Create header buttons for sorting - use dark background
    for col_name, width in add_columns:
        arrow = ''
        if add_state['sort_col'] == col_name:
            arrow = ' ↑' if add_state['sort_asc'] else ' ↓'
        
        btn = Button(description=f'{col_name}{arrow}')
        btn.layout = Layout(width=width, height='36px', margin='0', padding='0')
        btn.style.button_color = '#333'
        btn.add_class('header-sort-button')
        btn.on_click(make_add_sort_handler(col_name))
        add_header_buttons.append(btn)
    
    def on_add_filter_change(change):
        add_state['filter'] = change['new']
        add_state['page'] = 0
        render_additional_table()
    
    def on_add_prev(b):
        add_state['page'] -= 1
        render_additional_table()
    
    def on_add_next(b):
        add_state['page'] += 1
        render_additional_table()
    
    add_filter_input.observe(on_add_filter_change, names='value')
    add_prev_btn.on_click(on_add_prev)
    add_next_btn.on_click(on_add_next)
    
    render_additional_table()
    
    add_title_widget = HTML(f'''
        <div style="
            background: linear-gradient(135deg, #3498db 0%, #2980b9 100%);
            color: white;
            padding: 20px;
            border-radius: 10px 10px 0 0;
        ">
            <h3 style="margin: 0; font-size: 18px;">Additional Studies</h3>
            <p style="margin: 5px 0 0 0; opacity: 0.9; font-size: 13px;">{extra_studies} additional studies found. {multi_study_accessions_count} accession numbers have multiple studies in the PACS</p>
        </div>
    ''')
    
    # Left side: search box
    add_left_controls = HBox([add_filter_input], layout=Layout(align_items='center'))
    
    # Right side: pagination
    add_right_controls = HBox([add_prev_btn, add_page_label, add_next_btn], layout=Layout(align_items='center', gap='10px'))
    
    # Controls row with space-between to push pagination to right
    add_controls = HBox(
        [add_left_controls, add_right_controls],
        layout=Layout(padding='15px 20px', justify_content='space-between', align_items='center', background='#f5f5f5', width='100%')
    )
    
    # Header row using HBox with buttons - dark background
    add_header_row = HBox(add_header_buttons)
    add_header_row.layout = Layout(width='100%', margin='0', padding='0', background='#333', overflow='hidden')
    add_header_row.add_class('no-gap')
    
    # Table output wrapper with explicit white background
    add_table_wrapper = VBox([add_table_output])
    add_table_wrapper.layout = Layout(background='white', padding='0', margin='0', width='100%', overflow='hidden')
    
    # Table container - header and table together with no gaps
    add_table_container = VBox([add_header_row, add_table_wrapper])
    add_table_container.layout = Layout(background='#333', padding='0', margin='0', width='100%', overflow='hidden')
    add_table_container.add_class('no-gap')
    
    add_full_widget = VBox([
        add_title_widget,
        add_controls,
        add_table_container,
    ])
    add_full_widget.layout = Layout(width='1500px', border_radius='10px', overflow='hidden', background='#f5f5f5', margin='10px 5px')
    add_full_widget.add_class('no-gap')
    
    display(add_full_widget)

In [ ]:
# Study Errors and Missing
import ipywidgets as widgets
from ipywidgets import HTML, HBox, VBox, Layout, Button, Output, Text
import html as html_module

def truncate_with_tooltip(text, max_len=50):
    """Truncate text and add tooltip if needed."""
    text = str(text)
    if len(text) <= max_len:
        return text
    truncated = text[:max_len].rsplit(' ', 1)[0] + '...'
    escaped_full = html_module.escape(text)
    escaped_truncated = html_module.escape(truncated)
    return f'<span title="{escaped_full}" style="cursor: help;">{escaped_truncated}</span>'

# CSS to fix button styling - force white text on dark background for both light/dark themes
button_style_css = HTML('''
<style>
.header-sort-button button,
.header-sort-button .widget-button,
.header-sort-button .jupyter-button,
.header-sort-button button .bp3-button-text,
.header-sort-button button span,
.header-sort-button button p {
    border-radius: 0 !important;
    font-weight: bold !important;
    font-size: 11px !important;
    text-transform: uppercase !important;
    border: none !important;
    border-bottom: 2px solid #555 !important;
    background-color: #333 !important;
    color: #fff !important;
}
.header-sort-button button:hover {
    background-color: #444 !important;
}
.no-gap {
    gap: 0 !important;
}
.no-gap > * {
    margin: 0 !important;
}
</style>
''')
display(button_style_css)

# Get XNAT host for building links
XNAT_HOST = os.environ.get('XNAT_HOST', '')

if not study_df.empty:
    # Recalculate counts
    errors = (study_df['status'] == 'ERROR').sum()
    rejected = (study_df['status'] == 'REJECTED').sum()
    
    # Find AVAILABLE studies missing from XNAT
    available_df = study_df[study_df['status'] == 'AVAILABLE'].copy()
    
    if 'experimentLabel' in available_df.columns:
        def is_missing_from_project(row):
            exp_label = row.get('experimentLabel')
            if pd.isna(exp_label) or exp_label == '' or exp_label is None:
                return True
            return str(exp_label).strip() not in xnat_exp_labels
        
        missing_from_project = available_df[available_df.apply(is_missing_from_project, axis=1)]
    else:
        missing_from_project = pd.DataFrame()
    
    missing_from_project_count = len(missing_from_project)
    
    # Build error details list
    failed_count = errors + rejected + missing_from_project_count
    if failed_count > 0:
        failed_df = study_df[study_df['status'].isin(['ERROR', 'REJECTED'])].copy()
        
        error_details = []
        
        for _, row in failed_df.iterrows():
            status = row.get('status', '')
            if status == 'ERROR':
                status_display = 'Error'
                reason = str(row.get('statusMessage', '')).strip() or 'Unknown'
            else:
                status_display = 'Rejected'
                reason = str(row.get('statusMessage', '')).strip() or 'Rejected by site'
            
            study_uid = str(row.get('studyInstanceUid', '')).strip()
            if not study_uid or study_uid == 'nan' or study_uid == 'None':
                study_uid = 'Unknown'
            
            exp_label = str(row.get('experimentLabel', '')).strip()
            if not exp_label or exp_label == 'nan' or exp_label == 'None':
                exp_label = 'Unknown'
            
            patient_id = str(row.get('patientId', '')).strip()
            if not patient_id or patient_id == 'nan' or patient_id == 'None':
                patient_id = 'Unknown'
            
            subject_label = str(row.get('subjectLabel', '')).strip()
            if not subject_label or subject_label == 'nan' or subject_label == 'None':
                subject_label = 'Unknown'
            
            error_details.append({
                'Experiment Label': str(exp_label),
                'Patient ID': str(patient_id),
                'Subject Label': str(subject_label),
                'Accession Number': str(row.get('accessionNumber', '')),
                'Study UID': study_uid,
                'Status': status_display,
                'Reason': reason,
            })
        
        for _, row in missing_from_project.iterrows():
            study_uid = str(row.get('studyInstanceUid', '')).strip()
            if not study_uid or study_uid == 'nan' or study_uid == 'None':
                study_uid = 'Unknown'
            
            exp_label = str(row.get('experimentLabel', '')).strip()
            if not exp_label or exp_label == 'nan' or exp_label == 'None':
                exp_label = 'Unknown'
            
            patient_id = str(row.get('patientId', '')).strip()
            if not patient_id or patient_id == 'nan' or patient_id == 'None':
                patient_id = 'Unknown'
            
            subject_label = str(row.get('subjectLabel', '')).strip()
            if not subject_label or subject_label == 'nan' or subject_label == 'None':
                subject_label = 'Unknown'
            
            error_details.append({
                'Experiment Label': str(exp_label),
                'Patient ID': str(patient_id),
                'Subject Label': str(subject_label),
                'Accession Number': str(row.get('accessionNumber', '')),
                'Study UID': study_uid,
                'Status': 'Missing',
                'Reason': 'Available study missing from Project',
            })
        
        error_table_df = pd.DataFrame(error_details)
        
        # ==================== ERROR DISTRIBUTION TABLE ====================
        if len(error_table_df) > 0:
            error_distribution = error_table_df.groupby('Reason').size().reset_index(name='Count')
            error_distribution = error_distribution.sort_values('Count', ascending=False).reset_index(drop=True)
            
            # Paginated error distribution
            DIST_PAGE_SIZE = 5
            dist_state = {'page': 0}
            dist_table_output = Output()
            dist_prev_btn = Button(description='Previous', button_style='info', layout=Layout(width='100px'))
            dist_next_btn = Button(description='Next', button_style='info', layout=Layout(width='100px'))
            dist_page_label = HTML()
            
            def render_dist_table():
                dist_table_output.clear_output(wait=True)
                total_pages = max(1, (len(error_distribution) + DIST_PAGE_SIZE - 1) // DIST_PAGE_SIZE)
                dist_state['page'] = max(0, min(dist_state['page'], total_pages - 1))
                start = dist_state['page'] * DIST_PAGE_SIZE
                end = start + DIST_PAGE_SIZE
                page_data = error_distribution.iloc[start:end]
                
                dist_page_label.value = f'<span style="font-size: 12px; color: #666;">Page {dist_state["page"] + 1} of {total_pages}</span>'
                dist_prev_btn.disabled = dist_state['page'] == 0
                dist_next_btn.disabled = dist_state['page'] >= total_pages - 1
                
                dist_rows = []
                for _, r in page_data.iterrows():
                    reason_text = html_module.escape(str(r['Reason']))
                    dist_rows.append(f'<tr><td style="padding: 8px 12px; border-bottom: 1px solid #eee; color: #333; text-align: left !important;">{reason_text}</td><td style="padding: 8px 12px; border-bottom: 1px solid #eee; text-align: center; font-weight: bold; width: 80px; color: #333;">{r["Count"]}</td></tr>')
                
                table_html = f'''<table style="width: 100%; border-collapse: collapse; font-size: 12px; background: white;">
                    <thead>
                        <tr style="background: #f8f9fa;">
                            <th style="padding: 10px 12px; text-align: left !important; font-size: 11px; text-transform: uppercase; color: #666; border-bottom: 2px solid #ddd;">Reason</th>
                            <th style="padding: 10px 12px; text-align: center; font-size: 11px; text-transform: uppercase; color: #666; border-bottom: 2px solid #ddd; width: 80px;">Count</th>
                        </tr>
                    </thead>
                    <tbody>{"".join(dist_rows)}</tbody>
                </table>'''
                with dist_table_output:
                    display(HTML(table_html))
            
            def on_dist_prev(b):
                dist_state['page'] -= 1
                render_dist_table()
            
            def on_dist_next(b):
                dist_state['page'] += 1
                render_dist_table()
            
            dist_prev_btn.on_click(on_dist_prev)
            dist_next_btn.on_click(on_dist_next)
            render_dist_table()
            
            dist_header = HTML(f'''
                <div style="
                    background: linear-gradient(135deg, #9b59b6 0%, #8e44ad 100%);
                    color: white;
                    padding: 20px;
                    border-radius: 10px 10px 0 0;
                ">
                    <h3 style="margin: 0; font-size: 18px;">Study Error Distribution</h3>
                    <p style="margin: 5px 0 0 0; opacity: 0.9; font-size: 13px;">{len(error_distribution)} unique error reasons</p>
                </div>
            ''')
            
            dist_pagination = HBox(
                [dist_prev_btn, dist_page_label, dist_next_btn],
                layout=Layout(padding='8px 12px', justify_content='flex-end', align_items='center', gap='10px', background='#f8f9fa', border_top='1px solid #eee', border_radius='0 0 10px 10px')
            )
            
            dist_widget = VBox([
                dist_header,
                dist_pagination,
                VBox([dist_table_output], layout=Layout(height='230px', overflow='hidden', background='white')),
            ], layout=Layout(width='1500px', border_radius='10px', overflow='hidden', background='#f5f5f5', margin='10px 5px'))
            
            display(dist_widget)
        
        # ==================== STUDY ERRORS TABLE WITH FILTER ====================
        PAGE_SIZE = 15
        state = {'page': 0, 'sort_col': 'Status', 'sort_asc': True, 'filter': ''}
        
        # Column definitions
        columns = [
            ('Experiment Label', '14%'),
            ('Patient ID', '9%'),
            ('Subject Label', '9%'),
            ('Accession Number', '10%'),
            ('Study UID', '30%'),
            ('Status', '6%'),
            ('Reason', '22%'),
        ]
        
        filter_input = Text(
            placeholder='Search',
            layout=Layout(width='300px')
        )
        
        prev_btn = Button(description='Previous', button_style='info', layout=Layout(width='100px'))
        next_btn = Button(description='Next', button_style='info', layout=Layout(width='100px'))
        page_label = HTML()
        table_output = Output()
        
        # Create header buttons list first (will be populated below)
        header_buttons = []
        
        def update_header_buttons():
            """Update header button labels with sort arrows."""
            for i, (col_name, width) in enumerate(columns):
                arrow = ''
                if state['sort_col'] == col_name:
                    arrow = ' ↑' if state['sort_asc'] else ' ↓'
                header_buttons[i].description = f'{col_name}{arrow}'
        
        def make_sort_handler(col_name):
            def handler(b):
                if state['sort_col'] == col_name:
                    state['sort_asc'] = not state['sort_asc']
                else:
                    state['sort_col'] = col_name
                    state['sort_asc'] = True
                state['page'] = 0
                update_header_buttons()
                render_table()
            return handler
        
        def render_table():
            table_output.clear_output(wait=True)
            
            # Apply filter across all columns
            filtered_df = error_table_df.copy()
            if state['filter']:
                filter_pattern = state['filter'].replace('*', '.*')
                mask = (
                    filtered_df['Experiment Label'].str.contains(filter_pattern, case=False, regex=True, na=False) |
                    filtered_df['Patient ID'].str.contains(filter_pattern, case=False, regex=True, na=False) |
                    filtered_df['Subject Label'].str.contains(filter_pattern, case=False, regex=True, na=False) |
                    filtered_df['Accession Number'].str.contains(filter_pattern, case=False, regex=True, na=False) |
                    filtered_df['Study UID'].str.contains(filter_pattern, case=False, regex=True, na=False) |
                    filtered_df['Status'].str.contains(filter_pattern, case=False, regex=True, na=False) |
                    filtered_df['Reason'].str.contains(filter_pattern, case=False, regex=True, na=False)
                )
                filtered_df = filtered_df[mask]
            
            sorted_df = filtered_df.sort_values(
                by=state['sort_col'], 
                ascending=state['sort_asc'],
                key=lambda x: x.str.lower() if x.dtype == 'object' else x
            ).reset_index(drop=True)
            
            total_pages = max(1, (len(sorted_df) + PAGE_SIZE - 1) // PAGE_SIZE)
            state['page'] = max(0, min(state['page'], total_pages - 1))
            start = state['page'] * PAGE_SIZE
            end = start + PAGE_SIZE
            page_df = sorted_df.iloc[start:end].copy()
            
            filter_text = f" (filtered: {len(filtered_df)})" if state['filter'] else ""
            page_label.value = f'<span style="font-size: 13px; color: #666;">Page {state["page"] + 1} of {total_pages} ({len(sorted_df)} issues{filter_text})</span>'
            
            prev_btn.disabled = state['page'] == 0
            next_btn.disabled = state['page'] >= total_pages - 1
            
            if len(page_df) > 0:
                # Build data rows only (header is buttons)
                rows = []
                for _, r in page_df.iterrows():
                    status = str(r['Status'])
                    if status == 'Error':
                        bg_color = '#fdeaea'
                        text_color = '#721c24'
                    elif status == 'Rejected':
                        bg_color = '#fff8e6'
                        text_color = '#856404'
                    else:  # Missing
                        bg_color = '#e8daef'
                        text_color = '#4a235a'
                    
                    exp_label = str(r['Experiment Label'])
                    pid = html_module.escape(str(r['Patient ID']))
                    slabel = html_module.escape(str(r['Subject Label']))
                    acc = html_module.escape(str(r['Accession Number']))
                    uid = html_module.escape(str(r['Study UID']))
                    stat = html_module.escape(str(r['Status']))
                    reason = str(r['Reason'])
                    reason_truncated = truncate_with_tooltip(reason, 60)
                    
                    # Create experiment label with link if it exists and study is in XNAT
                    if exp_label and exp_label != 'Unknown' and exp_label.strip() in xnat_exp_labels:
                        xnat_url = f"{XNAT_HOST}/data/archive/projects/{PROJECT_ID}/experiments/{exp_label.strip()}"
                        exp_display = f'<a href="{xnat_url}" target="_blank" style="color: {text_color}; text-decoration: underline;">{html_module.escape(exp_label)}</a>'
                    else:
                        exp_display = html_module.escape(exp_label)
                    
                    rows.append(f'''<tr style="background-color: {bg_color}; color: {text_color};">
                        <td style="padding: 8px 12px; border-bottom: 1px solid #eee; width: 14%;">{exp_display}</td>
                        <td style="padding: 8px 12px; border-bottom: 1px solid #eee; width: 9%;">{pid}</td>
                        <td style="padding: 8px 12px; border-bottom: 1px solid #eee; width: 9%;">{slabel}</td>
                        <td style="padding: 8px 12px; border-bottom: 1px solid #eee; width: 10%;">{acc}</td>
                        <td style="padding: 8px 12px; border-bottom: 1px solid #eee; width: 30%; word-break: break-all;">{uid}</td>
                        <td style="padding: 8px 12px; border-bottom: 1px solid #eee; width: 6%;">{stat}</td>
                        <td style="padding: 8px 12px; border-bottom: 1px solid #eee; width: 22%;">{reason_truncated}</td>
                    </tr>''')
                
                table_html = f'''<table style="width: 100%; border-collapse: collapse; font-size: 12px; table-layout: fixed; background: white; margin: 0; padding: 0;">
                    <tbody>{"".join(rows)}</tbody>
                </table>'''
                
                # Unescape tooltips
                table_html = table_html.replace('&lt;span title=', '<span title=')
                table_html = table_html.replace('style=&quot;cursor: help;&quot;&gt;', 'style="cursor: help;">')
                table_html = table_html.replace('&lt;/span&gt;', '</span>')
                table_html = table_html.replace('&quot;', '"')
                
                with table_output:
                    display(HTML(table_html))
            else:
                with table_output:
                    display(HTML('<div style="padding: 20px; text-align: center; color: #666; background: white;">No matching studies found.</div>'))
        
        # Create header buttons for sorting - use dark background
        for col_name, width in columns:
            arrow = ''
            if state['sort_col'] == col_name:
                arrow = ' ↑' if state['sort_asc'] else ' ↓'
            
            btn = Button(description=f'{col_name}{arrow}')
            btn.layout = Layout(width=width, height='36px', margin='0', padding='0')
            btn.style.button_color = '#333'
            btn.add_class('header-sort-button')
            btn.on_click(make_sort_handler(col_name))
            header_buttons.append(btn)
        
        def on_filter_change(change):
            state['filter'] = change['new']
            state['page'] = 0
            render_table()
        
        def on_prev(b):
            state['page'] -= 1
            render_table()
        
        def on_next(b):
            state['page'] += 1
            render_table()
        
        filter_input.observe(on_filter_change, names='value')
        prev_btn.on_click(on_prev)
        next_btn.on_click(on_next)
        
        render_table()
        
        title_widget = HTML(f'''
            <div style="
                background: linear-gradient(135deg, #e74c3c 0%, #c0392b 100%);
                color: white;
                padding: 20px;
                border-radius: 10px 10px 0 0;
            ">
                <h3 style="margin: 0; font-size: 18px;">Missing Studies</h3>
                <p style="margin: 5px 0 0 0; opacity: 0.9; font-size: 13px;">{errors} errors, {rejected} rejected, {missing_from_project_count} missing from project</p>
            </div>
        ''')
        
        # Left side: search box
        left_controls = HBox([filter_input], layout=Layout(align_items='center'))
        
        # Right side: pagination
        right_controls = HBox([prev_btn, page_label, next_btn], layout=Layout(align_items='center', gap='10px'))
        
        # Controls row with space-between to push pagination to right
        controls = HBox(
            [left_controls, right_controls],
            layout=Layout(padding='15px 20px', justify_content='space-between', align_items='center', background='#f5f5f5', width='100%')
        )
        
        # Header row using HBox with buttons - add class for CSS targeting
        header_row = HBox(header_buttons)
        header_row.layout = Layout(width='100%', margin='0', padding='0', background='#333', overflow='hidden')
        header_row.add_class('no-gap')
        
        legend = HTML('''
            <div style="
                padding: 10px 20px;
                font-size: 11px;
                color: #666;
                background: #f8f9fa;
                border-top: 1px solid #eee;
                border-radius: 0 0 10px 10px;
            ">
                <span style="background: #fdeaea; padding: 2px 8px; border-radius: 3px; margin-right: 15px; color: #721c24;">Error</span> Failed to process
                <span style="margin-left: 20px; background: #fff8e6; padding: 2px 8px; border-radius: 3px; margin-right: 15px; color: #856404;">Rejected</span> Rejected by site
                <span style="margin-left: 20px; background: #e8daef; padding: 2px 8px; border-radius: 3px; margin-right: 15px; color: #4a235a;">Missing</span> Available study missing from project
            </div>
        ''')
        
        # Table output wrapper with explicit white background
        table_wrapper = VBox([table_output])
        table_wrapper.layout = Layout(background='white', padding='0', margin='0', width='100%', overflow='hidden')
        
        # Table container - header and table together with no gaps
        table_container = VBox([header_row, table_wrapper])
        table_container.layout = Layout(background='#333', padding='0', margin='0', width='100%', overflow='hidden')
        table_container.add_class('no-gap')
        
        full_widget = VBox([
            title_widget,
            controls,
            table_container,
            legend
        ])
        full_widget.layout = Layout(width='1500px', border_radius='10px', overflow='hidden', background='#f5f5f5', margin='10px 5px')
        full_widget.add_class('no-gap')
        
        display(full_widget)
    else:
        display(HTML('''
            <div style="
                background: #d4edda;
                border: 1px solid #28a745;
                border-radius: 6px;
                padding: 15px 20px;
                margin: 10px 5px;
                font-size: 13px;
                color: #155724;
                width: 1500px;
                box-sizing: border-box;
            ">
                <strong>All studies processed successfully.</strong> No errors, rejections, or missing studies.
            </div>
        '''))

In [ ]:
# Missing Series Analysis (displayed at bottom after series data is loaded)
if missing_series_output is not None:
    display(missing_series_output)

In [ ]:
session.disconnect()